## Interpolation of FKtable x-grid

In [ ]:
from validphys.coredata import FKTableData
from validphys.api import API
from validphys.fkparser import load_fktable
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d, lagrange
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

### Simple Example of Interpolation using scipy.interpolate

In [2]:
x = np.array([1,2,3])
y = x
# default `kind` for spline interpolator is linear
f = interp1d(x, y, fill_value="extrapolate")
x_new = np.linspace(0.1,1,10)

f(x_new) - x_new

array([-2.77555756e-17, -5.55111512e-17,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00])

In [3]:
# Example with NaNs
x = np.array([np.nan,np.nan,1,2,3])
y = np.array([-1,0,1,2,3])

# default `kind` for spline interpolator is linear
f = interp1d(x, y, fill_value="extrapolate")
x_new = np.linspace(0.1,1,10)

f(x_new) - x_new

array([-2.77555756e-17, -5.55111512e-17,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00])

In [4]:
inp = {
    "fit": "210713-n3fit-001",
    "dataset_inputs": {"from_": "fit"},
    "use_cuts": "internal",
    "theoryid": 200
}

inp_400 = {
    "fit": "210713-n3fit-001",
    "dataset_inputs": {"from_": "fit"},
    "use_cuts": "internal",
    "theoryid": 400
}

In [5]:
data = API.data(**inp)
data_400 = API.data(**inp_400)

In [27]:
datasets = data.datasets
datasets_400 = data_400.datasets

fktable_dis = load_fktable(datasets[0].fkspecs[0])
fktable_dis_400 = load_fktable(datasets_400[0].fkspecs[0])

new_xgrid = fktable_dis_400.xgrid

### Example for one datapoint

In [25]:
# incomplete dataframe
df_incomplete = fktable_dis.sigma[fktable_dis.sigma.index.get_level_values('data') == 31]

# Create an empty dataframe with the full xgrid range
xgrid = fktable_dis.xgrid
xgrid_new = fktable_dis_400.xgrid

full_df = pd.DataFrame({'x': xgrid, 'x_new':xgrid_new})

# Use .loc to fill in the values from the incomplete dataframe
full_df.loc[df_incomplete.index.get_level_values('x'), df_incomplete.columns] = df_incomplete[df_incomplete.columns].values

f_1_lin = interp1d(full_df.dropna().x , full_df.dropna()[1], fill_value="extrapolate")
f_1_quad = interp1d(full_df.dropna().x, full_df.dropna()[1], kind='quadratic', fill_value="extrapolate")
f_1_cube = interp1d(full_df.dropna().x, full_df.dropna()[1], kind='cubic', fill_value="extrapolate")
full_df['1_new_lin'] = f_1_lin(xgrid_new)
# full_df['1_new_quad'] = f_1_quad(xgrid_new)
# full_df['1_new_cube'] = f_1_cube(xgrid_new)

In [26]:
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=full_df.x, y=full_df[1],
                    mode='lines+markers',
                    name='Theory 200 grid'))

fig.add_trace(go.Scatter(x=fktable_dis_400.xgrid, y=fktable_dis_400.sigma[1],
                    mode='lines+markers',
                    name='Theory 400 grid'))

fig.add_trace(go.Scatter(x=full_df.x_new, y=full_df['1_new_lin'],
                    mode='lines+markers',
                    name='Linear Spline Interpolation'))

# fig.add_trace(go.Scatter(x=full_df.x_new, y=full_df['1_new_quad'],
#                     mode='lines+markers',
#                     name='Quadratic Spline Interpolation'))

# fig.add_trace(go.Scatter(x=full_df.x_new, y=full_df['1_new_cube'],
#                     mode='lines+markers',
#                     name='Cubic Spline Interpolation'))

fig.show()

### Example for 1 FKTableData corresponding to 1 fkspec of a dataset

In [36]:
a = lambda x,y: x+y
a(1,2)

3

In [34]:
fktable_dis.sigma

linear_interp = lambda x,y: interp1d(x, y, fill_value="extrapolate")
columns_interpolators = []

# loop over datapoints
for d, grp in fktable_dis.sigma.groupby(fktable_dis.sigma.index.get_level_values('data')):
    pass

1         2         10        11
data x                                         
31   6   0.013610 -0.001051  0.004148  0.002985
     7   0.328438 -0.026199  0.100113  0.072159
     8  -0.317752 -0.001202 -0.094918 -0.061420
     9   0.409540  0.014364  0.050562 -0.207163
     10 -0.197674  0.001586 -0.012371  0.148052
...           ...       ...       ...       ...
259  45  0.007404  0.000703  0.000884 -0.003871
     46  0.004211  0.000850  0.000674 -0.001512
     47  0.004492  0.000960  0.000990 -0.000526
     48  0.004146  0.001043  0.000815 -0.000876
     49  0.003691  0.001106  0.000717 -0.000807

[4425 rows x 4 columns]